# Create the Dataset
This has to be done only once or if the dataset is modified

In [ ]:
# Load path/class_id image file:
data_folder = 'data/pren_dataset_large'

# Build a HDF5 dataset (only required once)
from tflearn.data_utils import build_hdf5_image_dataset
build_hdf5_image_dataset(data_folder, image_shape=(128, 128), mode='folder', output_path='data/dataset_large.h5', categorical_labels=True, normalize=True, grayscale=True)

# Imports

In [1]:
import numpy as np
from tflearn import DNN
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

# Load the Dataset

In [34]:
import h5py
h5f = h5py.File('data/dataset_large.h5', 'r')
X = h5f['X']
Y = h5f['Y']

## Check the datashape

In [35]:
print(X[1].shape)
X = np.array(X).reshape([-1, 128, 128, 1])
print(X[50].shape)

(128, 128)
(128, 128, 1)


In [36]:
print(Y[1].shape)
print(Y[1])

(5,)
[ 1.  0.  0.  0.  0.]


# Train the Deep Neural Network

In [ ]:
network = input_data(shape=[None, 128, 128, 1])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

model = DNN(network, tensorboard_verbose=3)

In [ ]:
model.fit(X, Y, n_epoch=5, shuffle=True, validation_set=0.2, show_metric=True, batch_size=128, validation_batch_size=128, run_id='model_prod')

model.save('models/prod_model')
h5f.close()

# Predict with picamera

In [ ]:
# from time import sleep
# import picamera
# camera = picamera.PiCamera()

# while (True):
#     camera.capture('image.jpg')
#     sleep(1)

# Load saved network

In [2]:
network = input_data(shape=[None, 128, 128, 1])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

model = DNN(network, tensorboard_verbose=0)

model.load('models/prod_model')

# Mesure the time it takes for 10 predictions

In [31]:
import time
start_time = time.time()

for i in range(1, 10):
    predication = model.predict([X[i]])
    print(np.round(predication, 2))

print("--- %s seconds ---" % (time.time() - start_time))

[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
[[ 1.  0.  0.  0.  0.]]
--- 0.34261298179626465 seconds ---
